In [8]:
from datasets import load_dataset
from transformers import pipeline

In [1]:
raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")

/home/ivo/Programming_Personal_Projects/NLP_exploration/.env_nlp_exploration/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

In [3]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [5]:
split_datasets["validation"] = split_datasets.pop("test")
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [6]:
split_datasets["train"][1]["translation"]

{'en': 'Default to expanded threads',
 'fr': 'Par défaut, développer les fils de discussion'}

## Importing The Model

In [16]:
model_checkpoint = "Helsinki-NLP/opus-mt-fr-en"
translator = pipeline("translation", model=model_checkpoint)

/home/ivo/Programming_Personal_Projects/NLP_exploration/.env_nlp_exploration/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


## A Simple Translation

In [10]:
french_text = """Aujourd'hui largement répandus, les tatouages ont vu leurs significations fluctuer dans 
    l'espace et dans le temps. De l'aristocratie anglaise au «Milieu», des marins aux 
    prostituées, tatoueurs et tatoués sont des marqueurs de leur temps, à travers les motifs 
    et les pratiques. Sur nos plages et dans nos rues, lorsqu’il fait chaud, une évidence 
    s’impose, celle d’un phénomène remarquable : la présence accrue de tatouages sur la peau 
    de nos contemporains. De savantes statistiques confirment cette impression empirique, sans
    qu’on puisse trouver facilement une explication à cette efflorescence. Elle est faite pour
    intriguer. Donc il est tentant d’aller rechercher dans le passé ce que peuvent être les 
    origines et le développement de la pratique du tatouage. 
    Maryan Guisy s’est attaché à déchiffrer le sens profond d’un comportement aussi universel 
    qu’énigmatique. Il démontre que la marque corporelle, qui remonte à la plus haute 
    Antiquité, est un mode d’expression constant de l’humanité, depuis le fond des grottes du 
    paléolithique jusqu’aux studios des artisans du métier les plus contemporains. Ces marques
    ont été souvent imposées par une communauté, que ce soit pour affirmer une appartenance, 
    pour signifier une servitude ou pour perpétuer une punition. L’empreinte du fer rouge sur 
    l’épaule de Milady, dans les Trois mousquetaires, flambe encore dans nos mémoires. 
    Mais les tatouages ont été aussi, de longue date, choisis librement par ceux et par 
    celles qui se les ont imprimés sur leur corps. Il s’est agi alors d’affirmer des 
    solidarités, corporatives, militaires ou de brigandage, propres à protéger l’individu 
    isolé contre les périls qui le menaceraient ou à l’inclure au cœur de bonheurs festifs 
    partagés. Et on n’oubliera pas d’évoquer la valeur de talisman que peuvent prendre, dans 
    une ambiance mystique, ces inscriptions délibérées sur la peau. Rien de simple, en somme, 
    mais vous connaissez le mantra des historiens : "c’est plus compliqué que vous ne le 
    pensiez"."""

In [11]:
splitted_french_text = french_text.split(".")

In [42]:
splitted_french_text

["Aujourd'hui largement répandus, les tatouages ont vu leurs significations fluctuer dans \n    l'espace et dans le temps",
 " De l'aristocratie anglaise au «Milieu», des marins aux \n    prostituées, tatoueurs et tatoués sont des marqueurs de leur temps, à travers les motifs \n    et les pratiques",
 ' Sur nos plages et dans nos rues, lorsqu’il fait chaud, une évidence \n    s’impose, celle d’un phénomène remarquable : la présence accrue de tatouages sur la peau \n    de nos contemporains',
 ' De savantes statistiques confirment cette impression empirique, sans\n    qu’on puisse trouver facilement une explication à cette efflorescence',
 ' Elle est faite pour\n    intriguer',
 ' Donc il est tentant d’aller rechercher dans le passé ce que peuvent être les \n    origines et le développement de la pratique du tatouage',
 ' \n    Maryan Guisy s’est attaché à déchiffrer le sens profond d’un comportement aussi universel \n    qu’énigmatique',
 ' Il démontre que la marque corporelle, qui rem

In [34]:
paragraphs = [ True if sentence.find("\n")!=-1 else False 
              for sentence in splitted_french_text ]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False]

In [39]:
translated_text = translator(splitted_french_text)
translated_text = [text["translation_text"] for text in translated_text]

In [40]:
translated_text = [sentence+".\n" for (sentence,paragraph) in zip(translated_text,paragraphs)
                   if paragraph]
translated_text = "".join(translated_text).replace("..",".")

In [43]:
print(translated_text)

Today widely distributed, tattoos have seen their meanings fluctuate in space and time.
From the English aristocracy to the "Milieu", sailors to prostitutes, tattooers and tattoos are markers of their time, through patterns and practices.
On our beaches and in our streets, when it's hot, there's evidence of a remarkable phenomenon: the increased presence of tattoos on the skin of our contemporaries.
Scientists confirm this empirical impression, without an easy explanation for this efflorescence.
It's made to intrigue.
So it's tempting to go look in the past what the origins and the development of tattoo practice can be.
Maryan Guisy is committed to deciphering the deep meaning of such universal and enigmatic behaviour.
It shows that the body brand, which dates back to the highest antiquity, is a constant way of expressing humanity, from the bottom of the caves of Paleolithic to the studios of the most contemporary artisans.
These marks have often been imposed by a community, whether to

## Fine-Tunning The Model

### TBD